<a href="https://colab.research.google.com/github/DarksAces/DarksAces/blob/main/Keytool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cryptography

In [9]:
!python /content/sample_data/mykeytool.py generar --keystore miClave.jks --alias miAlias --password MiPass123 --keysize 2048 --cn "Juan Pérez" --ou "Desarrollo" --o "Mi Empresa S.L." --l "Madrid" --s "Madrid" --c "ES"

[ERROR] El alias 'miAlias' ya existe en el keystore 'miClave.jks'.


In [6]:
!python /content/sample_data/mykeytool.py exportar --keystore miClave.jks --alias miAlias --password MiPass123 --archivo miCertificado.crt

[OK] Certificado exportado correctamente a 'miCertificado.crt'.


In [5]:
import subprocess
import os
import argparse
import sys

# ===================== Funciones =====================

def verificar_alias_existe(alias, keystore, password):
    """Verifica si un alias ya existe en el keystore"""
    if not os.path.exists(keystore):
        return False
    try:
        comando = [
            "keytool",
            "-list",
            "-alias", alias,
            "-keystore", keystore,
            "-storepass", password
        ]
        resultado = subprocess.run(comando, capture_output=True, text=True)
        return resultado.returncode == 0
    except Exception:
        return False

def listar_alias_keystore(keystore, password):
    """Lista todos los alias en un keystore"""
    try:
        comando = [
            "keytool",
            "-list",
            "-keystore", keystore,
            "-storepass", password
        ]
        resultado = subprocess.run(comando, capture_output=True, text=True, check=True)
        return True
    except subprocess.CalledProcessError:
        return False

def generar_keystore(alias, keystore, password, dname, keysize=2048):
    """Genera un nuevo keystore con un par de claves"""
    try:
        if os.path.exists(keystore):
            if verificar_alias_existe(alias, keystore, password):
                print(f"[ERROR] El alias '{alias}' ya existe en el keystore '{keystore}'.")
                return False
        comando = [
            "keytool",
            "-genkeypair",
            "-keyalg", "RSA",
            "-alias", alias,
            "-keystore", keystore,
            "-storepass", password,
            "-keypass", password,
            "-keysize", str(keysize),
            "-dname", dname
        ]
        subprocess.run(comando, check=True, capture_output=True)
        print(f"[OK] Keystore '{keystore}' generado correctamente con alias '{alias}'.")
        return True
    except subprocess.CalledProcessError as e:
        print("[ERROR] Error al ejecutar keytool:", e.stderr.decode() if e.stderr else str(e))
        return False
    except FileNotFoundError:
        print("[ERROR] keytool no encontrado. Asegúrate de tener Java JDK instalado.")
        return False

def exportar_certificado(alias, keystore, password, archivo_cert):
    """Exporta el certificado desde el keystore"""
    try:
        if not verificar_alias_existe(alias, keystore, password):
            print(f"[ERROR] El alias '{alias}' no existe en el keystore '{keystore}'.")
            return False
        if os.path.exists(archivo_cert):
            print(f"[INFO] El archivo '{archivo_cert}' ya existe. Será sobrescrito.")
        comando = [
            "keytool",
            "-export",
            "-alias", alias,
            "-file", archivo_cert,
            "-keystore", keystore,
            "-storepass", password
        ]
        subprocess.run(comando, check=True, capture_output=True)
        print(f"[OK] Certificado exportado correctamente a '{archivo_cert}'.")
        return True
    except subprocess.CalledProcessError as e:
        print("[ERROR] Error al exportar certificado:", e.stderr.decode() if e.stderr else str(e))
        return False
    except FileNotFoundError:
        print("[ERROR] keytool no encontrado. Asegúrate de tener Java JDK instalado.")
        return False

# ===================== CLI con argparse =====================

def main():
    parser = argparse.ArgumentParser(description="Herramienta para gestionar keystores y certificados.")
    subparsers = parser.add_subparsers(dest="comando", required=True)

    # Subcomando: generar
    gen_parser = subparsers.add_parser("generar", help="Generar un keystore con un alias")
    gen_parser.add_argument("--keystore", required=True, help="Nombre del keystore .jks")
    gen_parser.add_argument("--alias", required=True, help="Alias para la clave")
    gen_parser.add_argument("--password", required=True, help="Contraseña del keystore y clave")
    gen_parser.add_argument("--keysize", type=int, default=2048, help="Tamaño de la clave (default: 2048)")
    gen_parser.add_argument("--cn", default="Desconocido", help="Nombre común CN")
    gen_parser.add_argument("--ou", default="Desconocido", help="Unidad organizativa OU")
    gen_parser.add_argument("--o", default="Desconocido", help="Organización O")
    gen_parser.add_argument("--l", default="Desconocido", help="Localidad L")
    gen_parser.add_argument("--s", default="Desconocido", help="Estado S")
    gen_parser.add_argument("--c", default="ES", help="Código de país C (2 letras)")

    # Subcomando: exportar
    exp_parser = subparsers.add_parser("exportar", help="Exportar certificado de un alias")
    exp_parser.add_argument("--keystore", required=True, help="Nombre del keystore .jks")
    exp_parser.add_argument("--alias", required=True, help="Alias de la clave")
    exp_parser.add_argument("--password", required=True, help="Contraseña del keystore")
    exp_parser.add_argument("--archivo", required=True, help="Archivo de salida .crt")

    args = parser.parse_args()

    if args.comando == "generar":
        dname = f"CN={args.cn}, OU={args.ou}, O={args.o}, L={args.l}, S={args.s}, C={args.c}"
        if os.path.exists(args.keystore) and verificar_alias_existe(args.alias, args.keystore, args.password):
            print(f"[ERROR] El alias '{args.alias}' ya existe en el keystore '{args.keystore}'.")
            sys.exit(1)
        generar_keystore(args.alias, args.keystore, args.password, dname, args.keysize)

    elif args.comando == "exportar":
        if not verificar_alias_existe(args.alias, args.keystore, args.password):
            print(f"[ERROR] El alias '{args.alias}' no existe en '{args.keystore}'.")
            sys.exit(1)
        exportar_certificado(args.alias, args.keystore, args.password, args.archivo)

if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] {generar,exportar} ...
colab_kernel_launcher.py: error: argument comando: invalid choice: '/root/.local/share/jupyter/runtime/kernel-b5746964-70a0-49fe-ab96-41c0255030c6.json' (choose from generar, exportar)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2188, in _parse_known_args
    stop_index = consume_positionals(start_index)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2141, in consume_positionals
    take_action(action, args)
  File "/usr/lib/python3.12/argparse.py", line 2003, in take_action
    argument_values = self._get_values(action, argument_strings)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2523, in _get_values
    self._check_value(action, value[0])
  File "/usr/lib/python3.12/argparse.py", line 2573, in _check_value
    raise ArgumentError(action, msg % args)
argparse.ArgumentError: argument comando: i

TypeError: object of type 'NoneType' has no len()

In [8]:
import unittest
from unittest.mock import patch, MagicMock
import os
import keystore_tool  # Aquí importa tu script renombrado como módulo

class TestKeystoreTool(unittest.TestCase):

    @patch("subprocess.run")
    @patch("os.path.exists")
    def test_verificar_alias_existe_true(self, mock_exists, mock_run):
        mock_exists.return_value = True
        mock_run.return_value.returncode = 0
        self.assertTrue(keystore_tool.verificar_alias_existe("alias", "keystore.jks", "pass"))

    @patch("subprocess.run")
    @patch("os.path.exists")
    def test_verificar_alias_existe_false(self, mock_exists, mock_run):
        mock_exists.return_value = True
        mock_run.return_value.returncode = 1
        self.assertFalse(keystore_tool.verificar_alias_existe("alias", "keystore.jks", "pass"))

    @patch("subprocess.run")
    @patch("os.path.exists")
    def test_generar_keystore(self, mock_exists, mock_run):
        mock_exists.return_value = False
        mock_run.return_value.returncode = 0
        result = keystore_tool.generar_keystore(
            alias="test",
            keystore="test.jks",
            password="pass",
            dname="CN=test, OU=test, O=test, L=test, S=test, C=ES",
            keysize=2048
        )
        self.assertTrue(result)

    @patch("subprocess.run")
    @patch("os.path.exists")
    def test_exportar_certificado(self, mock_exists, mock_run):
        mock_exists.side_effect = [True, False]  # keystore existe, archivo no existe
        mock_run.return_value.returncode = 0
        result = keystore_tool.exportar_certificado("alias", "keystore.jks", "pass", "cert.crt")
        self.assertTrue(result)

if __name__ == "__main__":
    unittest.main()


ModuleNotFoundError: No module named 'keystore_tool'